In [1]:
import numpy as np
import scipy.io
from sklearn import metrics
import pandas as pd
import os
# os.environ['THEANO_FLAGS'] = "device=cuda0,force_device=True,floatX=float32"
# import theano
# print(theano.config.device)

from keras.layers import Embedding
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Layer, Input, Concatenate, Reshape
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.pooling import GlobalMaxPooling1D
from keras.layers import LSTM
from keras.layers import Bidirectional, TimeDistributed
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
def get_auroc(preds, obs):
    fpr, tpr, thresholds  = metrics.roc_curve(obs, preds, drop_intermediate=False)
    auroc = metrics.auc(fpr,tpr)
    return auroc

def get_aupr(preds, obs):
    precision, recall, thresholds  = metrics.precision_recall_curve(obs, preds)
    aupr = metrics.auc(recall,precision)
    return aupr

def get_aurocs_and_auprs(tpreds, tobs):
    tpreds_df = pd.DataFrame(tpreds)
    tobs_df = pd.DataFrame(tobs)
    
    task_list = []
    auroc_list = []
    aupr_list = []
    for task in tpreds_df:
        pred = tpreds_df[task]
        obs = tobs_df[task]
        auroc=round(get_auroc(pred,obs),5)
        aupr = round(get_aupr(pred,obs),5)
        task_list.append(task)
        auroc_list.append(auroc)
        aupr_list.append(aupr)
    return auroc_list, aupr_list

### Load data (test)


In [3]:
data_folder = "./deepsea_train/"

testmat = scipy.io.loadmat(data_folder+'test.mat')

### Load model

In [4]:
model = load_model("./model/tbinet.05-0.06.hdf5")
print('model summary')
model.summary()

2022-10-13 21:03:57.403185: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


model summary
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1000, 4)]         0         
                                                                 
 conv1d_8 (Conv1D)           (None, 975, 320)          33600     
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 75, 320)          0         
 1D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 75, 320)           0         
                                                                 
 bidirectional_6 (Bidirectio  (None, 75, 640)          1640960   
 nal)                                                            
                                                                 
 dropout_13 (Dropout)        (None, 75, 640) 

### Calculate averaged AUROC and AUPR

In [ ]:
test_transposed = np.transpose(testmat['testxdata'],axes=(0,2,1))
test_sub = test_transposed[:1000,125:815,:]
test_sub1 = test_transposed[1000:1001,125:815,:]
test_sub1.shape

In [22]:
x = np.transpose(testmat['testxdata'],axes=(0,2,1))[9990:9991,:,:]
result = model.predict(x)
result

1/1 [==============================] - 0s 39ms/step


array([[4.16350318e-03, 3.56986769e-03, 4.96459287e-03, 5.98314182e-05,
        5.63691401e-06, 5.87144447e-03, 2.27183546e-03, 1.24734826e-04,
        1.50035555e-03, 1.04052406e-02, 3.81200221e-06, 3.19734863e-05,
        5.33751352e-03, 4.96992422e-03, 3.68594914e-03, 7.05450680e-03,
        6.29185187e-03, 8.51538334e-06, 5.89465536e-03, 1.23998732e-04,
        8.86403769e-03, 5.48501778e-03, 3.11387485e-05, 3.32811731e-03,
        1.50680216e-02, 4.56176320e-04, 6.20876113e-03, 5.79900155e-03,
        8.37785774e-06, 4.77548238e-05, 8.05204082e-03, 4.23389627e-03,
        2.64958455e-03, 3.81802884e-03, 3.36103956e-03, 5.16962912e-03,
        4.44996491e-04, 8.76849331e-03, 3.87335196e-03, 5.79721155e-03,
        2.08634920e-02, 7.19372620e-05, 5.69326617e-03, 5.87811973e-03,
        5.08483732e-03, 6.35245524e-05, 5.05362696e-04, 1.00664701e-03,
        1.20766775e-03, 1.27548608e-03, 1.25883453e-04, 2.71891389e-04,
        2.99344538e-04, 2.10117123e-05, 1.37693627e-04, 4.131441

In [11]:
def softmax(x):
    """ 
    It return softmax values for each sets of scores in x
    """

    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [23]:
softmax(result[:,125:815])

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 

In [ ]:
tpreds = model.predict(np.transpose(testmat['testxdata'],axes=(0,2,1)),verbose=1)
tpreds_temp = np.copy(tpreds)
reverse_start_id = int(testmat['testdata'][:,125:815].shape[0]/2)

for i in range(reverse_start_id):
    tpreds_avg_temp = (tpreds_temp[i] + tpreds_temp[reverse_start_id+i])/2.0
    tpreds_temp[i] = tpreds_avg_temp
    tpreds_temp[reverse_start_id+i] = tpreds_avg_temp

aurocs, auprs = get_aurocs_and_auprs(tpreds_temp,testmat['testdata'][:,125:815])
print("Averaged AUROC:",np.nanmean(aurocs))
print("Averaged AUPR:", np.nanmean(auprs))